# Labelling of pole-like objects
E.g. trees, street lights, traffic signs.

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time

import src.fusion as fusion
import src.region_growing as growing
import src.utils.las_utils as las_utils
import src.utils.csv_utils as csv_utils
import src.utils.ahn_utils as ahn_utils
from src.pipeline import Pipeline
from src.labels import Labels

## Set-up
First, download and parse the required BGT data in notebook [1. Generate reference data](1.%20Generate%20reference%20data.ipynb). 

In [ ]:
# Select the point cloud file to process.
tilecode = '2386_9702'

in_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'
out_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'

# The corresponding BGT data file.
bgt_data_file = '../datasets/bgt/custom_pole_points_demo.csv'

# We need elevation data to determine object height above ground.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(data_folder=ahn_data_folder)

In [ ]:
# Load a (partially labelled) LAS file, e.g. from the previous step.
las = las_utils.read_las(in_file)

labels = las.label
points = np.vstack((las.x, las.y, las.z)).T

## Match point objects with clusters

First we try to match all objects in the BGT data with pole-like objects in the point cloud. We look for clusters of points with specific characteristics, such as a minimum height, density, and a maximum radius.

We start with trees, since the location data for these is most precise (in Amsterdam).

In [ ]:
# All possible parameters:
# params = {'search_pad': 1.5,   # Specify the padding (in m) around the BGT object in which to search for a match in the point cloud.
#           'max_dist': 1.2,     # Maximum distance (in m) between the expected location and the location of a potential match.
#           'voxel_res': 0.2,    # Resolution of voxels used when searching for a match.
#           'seed_height': 1.75, # Height above ground at which to determine object dimensions.
#           'min_height': 2.,    # Minimum hieght for an object to be considered a match.
#           'max_r': 0.5,        # Maximum radius for a pole-like object to be considered a match.
#           'min_points': 500,   # Minimum number of points for a cluster to be considered.
#           'z_min': 0.2,        # Height above ground, above which to search for objects.
#           'z_max': 2.7,        # Height above ground, below which to search for objects.
#           'r_mult': 1.5,       # Multiplier for radius when performing the initial (cylinder-based) labelling.
#           'label_height': 4.}  # Maximum height for initial (cylinder-based) labelling.

# Object type-specific parameters.
tree_params = {'seed_height': 1.75, 'min_points': 500, 'max_r': 0.5, 'label_height': 5.}
light_params = {'seed_height': 2.25, 'min_points': 400, 'max_r': 0.2, 'label_height': 5.}
sign_params = {'seed_height': 1.75, 'min_points': 200, 'max_r': 0.2, 'min_height': 1.2, 'z_max': 2., 'label_height': 3.}

# Create fusers for BGT point data.
tree_fuser = fusion.BGTPoleFuser(Labels.TREE, bgt_type='boom', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=tree_params)
light_fuser = fusion.BGTPoleFuser(Labels.STREET_LIGHT, bgt_type='lichtmast', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=light_params)
sign_fuser = fusion.BGTPoleFuser(Labels.TRAFFIC_SIGN, bgt_type='verkeersbord', bgt_file=bgt_data_file, ahn_reader=ahn_reader, params=sign_params)

In [ ]:
# Process the point cloud.
exclude_labels = (Labels.GROUND, Labels.BUILDING, Labels.CAR)
process_sequence = (tree_fuser, light_fuser, sign_fuser)
pipeline = Pipeline(process_sequence, exclude_labels, caching=False)

start = time.time()
labels = pipeline.process_cloud(tilecode, points, labels)
end = time.time()
print(f'Fusion completed in {end-start:.2f} seconds.\n')

## Extend point objects using LCC region growing

In the initial labelling phase we only labelled a cylinder-based region of the point cloud at the object's location.

We now refine the initial label by adding connected components that overlap with the initial label. This is done in two parts: close to the ground we want to be more careful, since there might be more clutter; while higher above ground we want to label the complete object without missing any part. Therefore, we use different region growing parameters for the top and bottom part of the object.

For more details on region growing, see notebook [5. Region growing](5.%20Region%20growing.ipynb)

This time we process the trees last, since these are harder to get right.

In [ ]:
# Each layer has the following (default) parameters:
# params = {'bottom': -inf,        # Height above ground at which the layer starts.
#           'top': inf,            # Height above ground at which the layer stops.
#           'grid_size': 0.1       # Octree grid size for LCC method, in meters. Lower means a more fine-grained clustering.
#           'min_comp_size': 100,  # Minimum number of points for a component to be considered.
#           'threshold': 0.5}      # Minimum fraction of points in a component that are already labelled initially for the component to be added.

# Parameter settings for the different layers.
light_bottom = {'top': 1.75, 'grid_size': 0.05, 'threshold': 0.5}
light_top = {'bottom': 1.75, 'grid_size': 0.1, 'threshold': 0.05}

sign_bottom = {'top': 1.5, 'grid_size': 0.05, 'threshold': 0.5}
sign_top = {'bottom': 1.5, 'grid_size': 0.1, 'threshold': 0.05}

tree_bottom = {'top': 1.75, 'grid_size': 0.05, 'threshold': 0.8}
tree_middle = {'bottom': 1.75, 'top':10.0, 'grid_size': 0.1, 'threshold': 0.01}
tree_top = {'bottom': 6.0, 'grid_size': 0.2, 'threshold': 0.01}

# Create the region growing objects.
light_grower = growing.LayerLCC(Labels.STREET_LIGHT, ahn_reader, 
                                params=[light_top, light_bottom])
sign_grower = growing.LayerLCC(Labels.TRAFFIC_SIGN, ahn_reader, 
                               params=[sign_top, sign_bottom])
tree_grower = growing.LayerLCC(Labels.TREE, ahn_reader, reset_noise=True,
                               params=[tree_bottom, tree_middle, tree_top])

In [ ]:
# Process the point cloud.
exclude_labels = (Labels.GROUND, Labels.BUILDING, Labels.CAR, Labels.TREE, Labels.STREET_LIGHT, Labels.TRAFFIC_SIGN)
process_sequence = (light_grower, sign_grower, tree_grower)
pipeline = Pipeline(process_sequence, exclude_labels, caching=False)

start = time.time()
labels = pipeline.process_cloud(tilecode, points, labels)
end = time.time()
print(f'Region growing completed in {end-start:.2f} seconds.\n')

### Save LAS file

In [ ]:
# Save the result (this will overwrite the LAS file.)
las_utils.label_and_save_las(las, labels, out_file)